## Ray Connet to local and remote server

### Configuration

In [2]:
! pip install --user -q "google-cloud-aiplatform[ray]>=1.56.0" \
                        "ray[data,train,tune,serve]>=2.33.0"

In [10]:
import ray
import datetime
import time
import torch
print(f"ray Version : {ray.__version__}")
print(f"torch Version : {torch.__version__}")

ray Version : 2.33.0
torch Version : 2.1.2+cu121


In [13]:
# Local CPU and GPU
import psutil
print(psutil.cpu_count())
print(torch.cuda.device_count())

48
4


### Attach to local ray server

In [14]:
ray.init(num_cpus= num_logical_cpus)

2025-02-16 11:49:28,254	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8267 


Python version:,3.10.16
Ray version:,2.33.0
Dashboard:,http://127.0.0.1:8267


(print_current_datetime pid=1213502) 2025-02-16 11:49:41.305254
(print_current_datetime pid=1213499) 2025-02-16 11:49:42.910457
(print_current_datetime pid=1213499) 2025-02-16 11:50:02.875808
(print_current_datetime pid=1213502) 2025-02-16 11:50:14.722452
(print_current_datetime pid=1213499) 2025-02-16 11:50:14.722462
(print_current_datetime pid=1213497) 2025-02-16 11:50:14.727226
(print_current_datetime pid=1213492) 2025-02-16 11:50:14.726777


In [15]:
# Ray Task    
@ray.remote
def print_current_datetime():
    time.sleep(3)
    current_datetime = datetime.datetime.now()
    print(current_datetime)
    return current_datetime

In [16]:
print_current_datetime.remote()

ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)

In [18]:
ray.get(print_current_datetime.remote())

datetime.datetime(2025, 2, 16, 11, 50, 2, 875808)

In [19]:
futures = [print_current_datetime.remote() for i in range(4)]
print(futures)

[ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000), ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000), ObjectRef(f4402ec78d3a2607ffffffffffffffffffffffff0100000001000000), ObjectRef(f91b78d7db9a6593ffffffffffffffffffffffff0100000001000000)]


In [20]:
ray.get(futures)


[datetime.datetime(2025, 2, 16, 11, 50, 14, 722462),
 datetime.datetime(2025, 2, 16, 11, 50, 14, 722452),
 datetime.datetime(2025, 2, 16, 11, 50, 14, 726777),
 datetime.datetime(2025, 2, 16, 11, 50, 14, 727226)]

In [21]:
ray.shutdown()

### Attach to remote Ray on Vertex AI

In [22]:
# @title Define constants
PROJECT_NBR = "721521243942"
PROJECT_ID = "ai-hangsik"
REGION="us-central1"
RAY_CLUSTER_NM = "ray33-cluster-20250216-192557"

In [23]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

In [24]:
import ray
from ray.runtime_env import RuntimeEnv
from ray.air.config import RunConfig
from ray.air import CheckpointConfig, ScalingConfig
from ray.util.joblib import register_ray

ray.__version__

'2.33.0'

In [25]:
ray.shutdown()

In [26]:
RAY_ADDRESS=f"vertex_ray://projects/{PROJECT_NBR}/locations/{REGION}/persistentResources/{RAY_CLUSTER_NM}"
print(f"RAY_ADDRESS:{RAY_ADDRESS}")

pip_env = {
  "pip": [
        "datasets==2.12.0",
        "evaluate==0.4.0",
        "accelerate==0.16.0",
        "transformers==4.26.0",
        "torch==1.13.0",
        "deepspeed==0.9.2",
        "ipython==8.14.0",
        "numpy<2.0.0",  # https://github.com/deepspeedai/DeepSpeed/issues/5671
        "python-json-logger"
  ],
}
ray.init(address=RAY_ADDRESS,runtime_env=pip_env)

RAY_ADDRESS:vertex_ray://projects/721521243942/locations/us-central1/persistentResources/ray33-cluster-20250216-192557
[Ray on Vertex AI]: Cluster State = State.RUNNING


SIGTERM handler is not set because current thread is not the main thread.


Python version:,3.10.16
Ray version:,2.33.0
Vertex SDK version:,1.80.0
Dashboard:,e8b73f504c907557-dot-us-central1.aiplatform-training.googleusercontent.com
Interactive Terminal Uri:,fe7820d66f7ca7e2-dot-us-central1.aiplatform-training.googleusercontent.com
Cluster Name:,ray33-cluster-20250216-192557


(print_current_datetime pid=1574, ip=10.127.0.100) 2025-02-16 12:48:03.826675
(print_current_datetime pid=1574, ip=10.127.0.100) 2025-02-16 12:48:09.400436
(print_current_datetime pid=1637, ip=10.127.0.100) 2025-02-16 12:48:10.390623
(print_current_datetime pid=1638, ip=10.127.0.100) 2025-02-16 12:48:10.392638
(print_current_datetime pid=1639, ip=10.127.0.100) 2025-02-16 12:48:10.429621


In [27]:
# Ray Task    
@ray.remote
def print_current_datetime():
    time.sleep(3)
    current_datetime = datetime.datetime.now()
    print(current_datetime)
    return current_datetime

In [28]:
ray.get(print_current_datetime.remote())

datetime.datetime(2025, 2, 16, 12, 48, 3, 826675)

In [29]:
futures = [print_current_datetime.remote() for i in range(4)]
ray.get(futures)

[datetime.datetime(2025, 2, 16, 12, 48, 9, 400436),
 datetime.datetime(2025, 2, 16, 12, 48, 10, 390623),
 datetime.datetime(2025, 2, 16, 12, 48, 10, 392638),
 datetime.datetime(2025, 2, 16, 12, 48, 10, 429621)]

In [30]:
ray.shutdown()